## Тема “Создание признакового пространства и NER - продолжение”

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [185]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

PREP_DATA = '../data/prep_tweets.pkl'

#### Загрузим подготовленный датасет твиттов

In [3]:
df_prep = pd.read_pickle(PREP_DATA)
df_prep.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [5]:
# получим текст из очищенных твитов
clean_tweets_text = " ".join(df_prep['clean_tweet'])
clean_tweets_text[:200]

'when father is dysfunctional and is so selfish he drags his kids into his dysfunction run thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked bihday'

### 1. Spacy

In [12]:
if 0:
    !pip install -U spacy
    !python -m spacy download en_core_web_md
    !python -m spacy info
    
import spacy
from spacy import displacy
import en_core_web_md


In [27]:
spacy_text = clean_tweets_text

In [29]:
%%time
nlp = en_core_web_md.load()
article = nlp(spacy_text[:1000])
displacy.render(article, jupyter=True, style='ent')

CPU times: user 6.58 s, sys: 342 ms, total: 6.92 s
Wall time: 6.9 s


In [30]:
# дерево зависимости
displacy.render(article[:10], style='dep', jupyter=True)

In [66]:
# слишком большой текст для spacy
len(spacy_text)

3455050

In [189]:
%%time
# применим NER распозновалку к каждому твиту
df_prep['spacy_ents'] = df_prep['clean_tweet'].progress_apply(lambda x: nlp(x).ents)

100%|██████████| 49159/49159 [14:32<00:00, 56.32it/s]

CPU times: user 14min 34s, sys: 5.25 s, total: 14min 39s
Wall time: 14min 32s


In [244]:
# Создадим массив ner с типом и преобразуем в датафрейм
def get_ner_dataframe(in_spacy_ents: pd.Series)-> pd.DataFrame: 
    word_list = []

    def add_to_word_list(ents):
        for ent in ents:
            word_list.append((ent.text, ent.label_))
        
    _ = in_spacy_ents.progress_apply(add_to_word_list)
    
    return pd.DataFrame(word_list, columns=['ner', 'ner_type'])

In [245]:
df_all_NERS = get_ner_dataframe(df_prep['spacy_ents'])
# df_train_NERS = get_ner_dataframe(df_prep.loc[df_prep.label < 1, 'spacy_ents'])
# df_test_NERS = get_ner_dataframe(df_prep.loc[df_prep.label == 1, 'spacy_ents'])

100%|██████████| 2242/2242 [00:00<00:00, 88647.42it/s]


### TOP 20 NER

In [233]:
# NER топ 20 общих
df_all_NERS.ner.value_counts()[:20]

today       1397
tomorrow     597
friday       588
one          518
first        514
orlando      473
sunday       466
morning      432
tonight      420
bihday       406
summer       402
saturday     323
monday       242
america      218
night        198
two          183
days         179
weekend      172
london       169
thursday     165
Name: ner, dtype: int64

In [320]:
# топ 20 трайн
df_all_NERS[df_all_NERS.ner_type == 'PERSON'].ner.value_counts()[:20]

bing bong bing bong                                        107
suppo                                                       69
obama                                                       67
hillary                                                     55
sta                                                         45
feminismiscancer feminismisterrorism feminismmuktbharat     40
melancholy melancholymusic                                  40
bjp                                                         36
christina grimmie                                           34
hu                                                          30
jo cox                                                      29
lebron                                                      26
tgif ff                                                     26
sea shepherd suppoers                                       25
donald trump                                                25
gamedev indiedev indiegamedev                          

In [235]:
# топ 20 тест
# df_test_NERS.ner.value_counts()[:20]

sjw                                                        73
allahsoil                                                  52
america                                                    51
obama                                                      34
tampa                                                      32
miami                                                      30
hispanic                                                   29
calgary                                                    27
sikh                                                       27
wso                                                        26
one                                                        21
daily                                                      20
feminismiscancer feminismisterrorism feminismmuktbharat    20
americans                                                  17
trump                                                      17
sea shepherd suppoers                                      17
nazi    

PERSON         11562
DATE           10821
ORG             8108
GPE             5415
TIME            2302
NORP            1701
CARDINAL        1079
ORDINAL          585
FAC              371
PRODUCT          326
LOC              284
EVENT            220
WORK_OF_ART      106
QUANTITY          55
LANGUAGE          40
MONEY             20
LAW               11
PERCENT            4
Name: ner_type, dtype: int64

### TOP 20 NER TYPES

In [236]:
df_all_NERS.ner_type.value_counts()[:20]

PERSON         11562
DATE           10821
ORG             8108
GPE             5415
TIME            2302
NORP            1701
CARDINAL        1079
ORDINAL          585
FAC              371
PRODUCT          326
LOC              284
EVENT            220
WORK_OF_ART      106
QUANTITY          55
LANGUAGE          40
MONEY             20
LAW               11
PERCENT            4
Name: ner_type, dtype: int64

In [238]:
# df_train_NERS.ner_type.value_counts()[:20]

DATE           7041
PERSON         6835
ORG            4617
GPE            3240
TIME           1488
NORP            708
CARDINAL        660
ORDINAL         365
FAC             225
PRODUCT         199
LOC             173
EVENT           127
WORK_OF_ART      62
QUANTITY         37
LANGUAGE         24
LAW               8
MONEY             6
PERCENT           3
Name: ner_type, dtype: int64

In [239]:
# df_test_NERS.ner_type.value_counts()[:20]

ORG            645
PERSON         628
GPE            375
NORP           374
DATE           134
CARDINAL        48
TIME            17
LOC             16
EVENT           14
ORDINAL         13
PRODUCT          7
WORK_OF_ART      5
LANGUAGE         4
FAC              4
MONEY            3
Name: ner_type, dtype: int64

### 2. NLTK

In [253]:
import nltk
if 0:
    nltk.download('averaged_perceptron_tagger')
    nltk.download('maxent_ne_chunker')
    nltk.download('words')

In [292]:
def nltk_ner(text:str)-> object:
    
    if not text:
        return None
    # тк зависит от регистра - переведем все в верхний
    ntxt = text.upper()
    nltk_chunks = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(ntxt)))
    return [(' '.join(c[0] for c in chunk), chunk.label()) for chunk in nltk_chunks if hasattr(chunk, 'label')]
    

In [293]:
%%time
# применим NER распозновалку к каждому твиту
df_prep['nltk_ents'] = df_prep['clean_tweet'].progress_apply(nltk_ner)

100%|██████████| 49159/49159 [06:44<00:00, 121.52it/s]

CPU times: user 6min 43s, sys: 4.49 s, total: 6min 48s
Wall time: 6min 44s


In [301]:
df_prep['nltk_ents'].head()

0       [(FATHER, ORGANIZATION), (INTO, ORGANIZATION)]
1    [(THANKS, ORGANIZATION), (FOR, ORGANIZATION), ...
2                                                   []
3    [(MODEL, ORGANIZATION), (LOVE, ORGANIZATION), ...
4    [(FACTSGUIDE, ORGANIZATION), (SOCIETY, ORGANIZ...
Name: nltk_ents, dtype: object

In [304]:
# Создадим массив ner с типом и преобразуем в датафрейм
def get_nltk_ner_dataframe(in_spacy_ents: pd.Series)-> pd.DataFrame: 
    word_list = []

    def add_to_word_list(ents):
        if not ents:
            return None
        
        for ent in ents:
            word_list.append((ent[0], ent[1]))
        
    _ = in_spacy_ents.progress_apply(add_to_word_list)
    
    return pd.DataFrame(word_list, columns=['ner', 'ner_type'])

In [305]:
df_nltk_ner = get_nltk_ner_dataframe(df_prep['nltk_ents'])

100%|██████████| 49159/49159 [00:00<00:00, 313391.50it/s]


In [307]:
df_nltk_ner.ner_type.value_counts()

ORGANIZATION    103397
GPE                877
PERSON             410
GSP                198
FACILITY            61
Name: ner_type, dtype: int64

In [317]:
df_nltk_ner[df_nltk_ner.ner_type == 'PERSON'].ner.value_counts()[:20]

IN                40
CLINTON           24
SORRY             21
CLICK             11
CLICK TO WATCH    11
CLICK RECIPE       7
JOHN               7
DE                 5
JOHNSON            5
TO THE             5
KIM                4
STEPHCURRY         4
PAUL               4
CURRY              4
US                 4
DAVID              3
ELECT              3
JOHN WOODEN        3
MILLER             3
STEPS              3
Name: ner, dtype: int64

## test2

In [308]:
# тк зависит от регистра - переведем все в верхний
nltk_text = clean_tweets_text.upper()

In [309]:
%%time
nltk_tags = nltk.pos_tag(nltk.word_tokenize(nltk_text))

CPU times: user 1min 20s, sys: 66.2 ms, total: 1min 20s
Wall time: 1min 20s


In [310]:
%%time
nltk_chunks = nltk.ne_chunk(nltk_tags)

CPU times: user 4min 48s, sys: 162 ms, total: 4min 48s
Wall time: 4min 48s


In [312]:
word_list = [(' '.join(c[0] for c in chunk), chunk.label()) for chunk in nltk_chunks if hasattr(chunk, 'label')]

In [313]:
df_nltk_ner1 = pd.DataFrame(word_list, columns=['ner', 'ner_type'])

In [314]:
df_nltk_ner1.ner_type.value_counts()

ORGANIZATION    65795
PERSON            269
GPE                40
FACILITY            3
GSP                 2
Name: ner_type, dtype: int64

In [319]:
df_nltk_ner1[df_nltk_ner.ner_type == 'PERSON'].ner.value_counts()[:20]

THE          19
HAPPINESS     4
TO            4
WORLD         3
WITH          3
CITY          3
NOT           2
THIS          2
BECAUSE       2
LIBTARD       2
LOOKS         2
YOUR          2
SO            2
THEY          2
ABOUT         2
AN            2
TWEETS        2
HARD          2
ORLANDO       2
FAMILY        2
Name: ner, dtype: int64

In [318]:
# %%time
# {(' '.join(c[0] for c in chunk), chunk.label()) \
#  for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(ntxt))) if hasattr(chunk, 'label') }

### 3. Какая из библиотек по вашему лучше отработала? 

Сравните качество полученных most_common NER и количество распознаных NER.



In [ ]:
from collections import defaultdict
word_freq = defaultdict(int)
for tokens in data.iloc[:]:
    for token in tokens:
        word_freq[token] += 1

In [ ]:
sorted(word_freq, key=word_freq.get, reverse=True)[:20]